In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict,GridSearchCV
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve 

In [2]:
data=pd.read_csv('../dataset/hmelq.csv')
df=data.copy() #data kopyalandı
df.dropna(inplace=True)# null veriler silindi
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2018 entries, 153 to 3549
Data columns (total 13 columns):
bad        2018 non-null int64
loan       2018 non-null int64
mortdue    2018 non-null float64
value      2018 non-null float64
reason     2018 non-null object
job        2018 non-null object
yoj        2018 non-null float64
derog      2018 non-null float64
delinq     2018 non-null float64
clage      2018 non-null float64
ninq       2018 non-null float64
clno       2018 non-null float64
debtinc    2018 non-null float64
dtypes: float64(9), int64(2), object(2)
memory usage: 220.7+ KB


In [3]:
df.head()

,bad,loan,mortdue,value,reason,job,yoj,derog,delinq,clage,ninq,clno,debtinc
153,0,18200,94727.0,136877.0,DebtCon,Mgr,15.0,0.0,0.0,168.96,2.0,26.0,36.056
154,0,21700,79240.0,96784.0,DebtCon,Mgr,5.0,0.0,0.0,64.51,6.0,24.0,38.079
155,0,34100,241931.0,36486.0,DebtCon,Mgr,1.0,0.0,2.0,196.01,3.0,50.0,42.459
156,0,8400,62989.0,76718.0,HomeImp,Mgr,3.0,0.0,2.0,131.47,0.0,22.0,29.200
157,0,17400,25859.0,43684.0,DebtCon,Mgr,16.0,1.0,0.0,95.36,1.0,17.0,27.108


In [4]:
df.reason.unique() #columns verilerini gösterir

array(['DebtCon', 'HomeImp'], dtype=object)

In [5]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
y=df["bad"]
X_=df.drop(["bad"],axis=1)#veri silme
# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
X = X_.apply(le.fit_transform) # kategorik veriler
X.head()

,loan,mortdue,value,reason,job,yoj,derog,delinq,clage,ninq,clno,debtinc
153,148,1534,1609,0,0,15,0,0,885,2,26,1098
154,183,1293,1068,0,0,5,0,0,64,6,24,1353
155,297,2000,27,0,0,1,0,2,1147,3,50,1817
156,50,886,611,1,0,3,0,2,632,0,22,478
157,140,143,73,0,0,16,1,0,276,1,17,342


In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

##  RandomTreeClassifier

In [7]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier().fit(X_train, y_train)
rf_model

/home/utkuglsvn/PycharmProjects/AnkaraMlBootcamp/venv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [8]:
y_pred=rf_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.9584158415841584

In [9]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       466
           1       0.95      0.49      0.64        39

    accuracy                           0.96       505
   macro avg       0.95      0.74      0.81       505
weighted avg       0.96      0.96      0.95       505



In [10]:
rf_params = {"max_depth": [2,5,7],
            "max_features": [2,5,8], 
            "n_estimators": [10,100,200],
            "min_samples_split": [2,5,10]}


rf_model = RandomForestClassifier()
rf_cv_model = GridSearchCV(rf_model,
                          rf_params,
                          cv=5,
                          n_jobs=-1,
                          verbose=2)

In [11]:
rf_cv_model.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:   46.1s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [12]:
print("en iyi parametre:"+str(rf_cv_model.best_params_))

en iyi parametre:{'max_depth': 7, 'max_features': 5, 'min_samples_split': 2, 'n_estimators': 100}


In [13]:
rfc=RandomForestClassifier(max_depth=7, max_features= 5, min_samples_split= 2, n_estimators=200)
rfc_tuned=rfc.fit(X_train,y_train)
rfc_tuned

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=7, max_features=5, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [14]:
y_pred=rfc_tuned.predict(X_test)
accuracy_score(y_test,y_pred) #acc skor

0.9524752475247524

In [15]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       466
           1       1.00      0.38      0.56        39

    accuracy                           0.95       505
   macro avg       0.98      0.69      0.77       505
weighted avg       0.95      0.95      0.94       505



##  DecisionTreeClassifier

In [16]:
from sklearn.tree import DecisionTreeClassifier
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)
dtc = DecisionTreeClassifier()
dtc_model = dtc.fit(X_train,y_train)

In [17]:
y_pred = dtc_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.9346534653465347

In [18]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96       466
           1       0.60      0.46      0.52        39

    accuracy                           0.93       505
   macro avg       0.78      0.72      0.74       505
weighted avg       0.93      0.93      0.93       505



In [19]:
dtc=DecisionTreeClassifier(max_depth=5,min_samples_split=19)
dtc_tuned=dtc.fit(X_train,y_train)
dtc_tuned

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=19,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [20]:
y_pred = dtc_tuned.predict(X_test)
accuracy_score(y_test, y_pred)

0.9465346534653465

##  GradientBoostingClassifier

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)
gbc = GradientBoostingClassifier()
gbc_model = gbc.fit(X_train,y_train)
gbc_model

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [22]:
gbm_model = GradientBoostingClassifier().fit(X_train, y_train)
y_pred = gbm_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.9584158415841584

In [23]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       466
           1       0.91      0.51      0.66        39

    accuracy                           0.96       505
   macro avg       0.93      0.75      0.82       505
weighted avg       0.96      0.96      0.95       505



In [24]:
gbm_params = {"learning_rate" : [0.001, 0.01, 0.1,0.05],
             "n_estimators": [100,500],
              "max_depth" : [3,5,5],
              "min_samples_split" : [2,5,5]}
gbm = GradientBoostingClassifier()

gbm_cv = GridSearchCV(gbm, gbm_params, cv = 5, n_jobs = -1, verbose =5)

In [25]:
gbm_cv.fit(X_train,y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   44.3s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  1.9min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
                                                  presort=

In [26]:
print("En iyi parametreler:" + str(gbm_cv.best_params_))

En iyi parametreler:{'learning_rate': 0.1, 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 500}


In [27]:
gbm = GradientBoostingClassifier(learning_rate = 0.05,
                                max_depth =3,
                                min_samples_split = 5,
                                n_estimators = 500)

In [28]:
gbm_tuned = gbm.fit(X_train, y_train)
y_pred = gbm_tuned.predict(X_test)
accuracy_score(y_test, y_pred)

0.9643564356435643

In [29]:
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98       466
           1       0.96      0.56      0.71        39

    accuracy                           0.96       505
   macro avg       0.96      0.78      0.85       505
weighted avg       0.96      0.96      0.96       505

